# Purpose: To compare the accuracy levels of a model trained with and without the augmented data set
- Steps:
    1. Build/reuse a LSTM based classification model, output: 1 out of 3 classes for each test sample.
    2. Use original dataset, train and test the accuracy level.
    3. Use augmented dataset (augmented rare classes + 80% of original data) for training, use 20% of original data for testing.
    4. Compare accuracy levels of 3 and 4. 